# Quick Start

**Home GitHub Repository:** [LEANN on GitHub](https://github.com/yichuan-w/LEANN)

**Important for Colab users:** Set your runtime type to T4 GPU for optimal performance. Go to Runtime → Change runtime type → Hardware accelerator → T4 GPU.

In [8]:
# install this if you are using colab
! uv pip install leann-core leann-backend-hnsw --no-deps
! uv pip install leann --no-deps
# For Colab environment, we need to set some environment variables
import os

os.environ["LEANN_LOG_LEVEL"] = "INFO"  # Enable more detailed logging

Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 36ms
Audited 2 packages in 0.40ms
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 1ms
Audited 1 package in 0.31ms


In [9]:
from pathlib import Path

INDEX_DIR = Path("./").resolve()
INDEX_PATH = str(INDEX_DIR / "demo.leann")

## Build the index

In [10]:
from leann.api import LeannBuilder

builder = LeannBuilder(backend_name="hnsw")
builder.add_text("C# is a powerful programming language and it is good at game development")
builder.add_text(
    "Python is a powerful programming language and it is good at machine learning tasks"
)
builder.add_text("Machine learning transforms industries")
builder.add_text("Neural networks process complex data")
builder.add_text("Leann is a great storage saving engine for RAG on your MacBook")
builder.build_index(INDEX_PATH)

INFO:leann.embedding_compute:Computing embeddings for 1 texts using SentenceTransformer, model: 'facebook/contriever'
INFO:leann.embedding_compute:Using cached optimized model: facebook/contriever
INFO:leann.embedding_compute:Starting embedding computation... (batch_size: 256, manual_tokenize=False)
INFO:leann.embedding_compute:Generated 1 embeddings, dimension: 768
INFO:leann.embedding_compute:Time taken: 0.01552891731262207 seconds
Writing passages: 100%|██████████| 5/5 [00:00<00:00, 7206.71chunk/s]
INFO:leann.embedding_compute:Computing embeddings for 5 texts using SentenceTransformer, model: 'facebook/contriever'
INFO:leann.embedding_compute:Using cached optimized model: facebook/contriever
INFO:leann.embedding_compute:Starting embedding computation... (batch_size: 256, manual_tokenize=False)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:leann.embedding_compute:Generated 5 embeddings, dimension: 768
INFO:leann.embedding_compute:Time taken: 0.04360079765319824 seconds


Starting conversion: /content/demo.index -> /content/demo.csr.tmp
[0.00s] Reading Index HNSW header...
[0.00s]   Header read: d=768, ntotal=5
[0.00s] Reading HNSW struct vectors...
  Reading vector (dtype=<class 'numpy.float64'>, fmt='d')... Count=6, Bytes=48
[0.00s]   Read assign_probas (6)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=7, Bytes=28
[0.43s]   Read cum_nneighbor_per_level (7)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=5, Bytes=20
[0.84s]   Read levels (5)
[1.26s]   Probing for compact storage flag...
[1.26s]   Found compact flag: False
[1.26s]   Compact flag is False, reading original format...
[1.26s]   Probing for potential extra byte before non-compact offsets...
[1.26s]   Found and consumed an unexpected 0x00 byte.
  Reading vector (dtype=<class 'numpy.uint64'>, fmt='Q')... Count=6, Bytes=48
[1.26s]   Read offsets (6)
[1.66s]   Attempting to read neighbors vector...
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Co

## Search with real-time embeddings

In [11]:
from leann.api import LeannSearcher

searcher = LeannSearcher(INDEX_PATH)
results = searcher.search("programming languages", top_k=2)
results

[SearchResult(id='0', score=np.float32(0.98745096), text='C# is a powerful programming language and it is good at game development', metadata={}),
 SearchResult(id='1', score=np.float32(0.8925245), text='Python is a powerful programming language and it is good at machine learning tasks', metadata={})]

## Chat with LEANN using retrieved results

In [15]:
from leann.api import LeannChat

llm_config = {
    "type": "hf",
    "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
}

chat = LeannChat(index_path=INDEX_PATH, llm_config=llm_config)
response = chat.ask(
    "Compare the two retrieved programming languages and tell me their advantages.",
    top_k=2,
    llm_kwargs={"max_tokens": 7000},
)
response

kwargs in HF:  {'max_tokens': 7000}


"Okay, so I need to compare C# and Python, but the context only mentions that C# is good for game development and Python for machine learning tasks. Hmm, that's a bit limited. I should think about what I know about each language beyond their main uses.\n\nStarting with C#: I remember it's used in game development, but there's more to it. It's an object-oriented language, which means classes and objects, which are pretty common in games. It has static methods and static classes, which can be a plus because they make code cleaner. Also, it's been around for a while, so it's stable. It's used in frameworks like Unity and Unity3D, which are essential for game development. So, advantages here might be its stability, extensive use in game development, and its strong community support.\n\nNow, Python: It's a general-purpose language, so it's used in many areas beyond ML. It's great for scripting, data analysis, web development, and automation. Python's flexibility and readability are big adva